![logo](IPUpy.png)

In [7]:
print('Hello world')

-------[5888x]-------
Hello world


In [8]:
print('I am tile', __tileid % 3)

-------[1963x]-------
I am tile 0

-------[1963x]-------
I am tile 1

-------[1962x]-------
I am tile 2


# Word Search

Each tile contains a fragment of the file sent to the IPU in special variable `__tiledata`. In this example we uploaded `/usr/share/dict/words`, i.e., the linux builin dictionary which is a newline-separated list of words. We can then operate on `__tiledata` in every interpreter, giving us an instantly parallelised interactive search scripting environment.

In [9]:
words = __tiledata.splitlines()
# print(words)

In [10]:
print([w for w in words if 'ipu' in w])

-------[1x]-------
['Itaipu']

-------[1x]-------
['Jaipur', "Jaipur's"]

-------[1x]-------
['Lilliput', "Lilliput's", 'Lilliputian', "Lilliputian's", 'Lilliputians']

-------[1x]-------
['Oedipus', "Oedipus's"]

-------[1x]-------
['manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', "manipulation's", 'manipulations', 'manipulativ']

-------[1x]-------
['manipulator', "manipulator's", 'manipulators']

-------[1x]-------
['multipurpose']

-------[1x]-------
['stipulate', 'stipulated', 'stipulates', 'stipulating', 'stipulation', "stipulation's", 'stipulations']

-------[5880x]-------
[]


# MandleBrot
For fun, we give each python interpreter control over 1 pixel using special function `setPixel`. If any tiles call this function, the notebook will display a visualisation of the 'colour' of every IPU tile. 

For example, if we visualise clock cycles, we sometimes glimpse the structure of the IPU in terms of relative latencies.

In [46]:
r = getTime()[0] % 256
g = getTime()[0] % 256
b = getTime()[0] % 256
setPixel(r, g, b)

With sufficient processors controlling 1 pixel each, we can generate some recognisable images!

In [1]:
# Find position of my pixel
H = [h for h in range(int(__numtiles ** 0.5), 0, -1) if __numtiles % h == 0][0]
W = __numtiles // H
X, Y = __tileid % W, __tileid // W

# Set animations consts
x_start = (X / W) * 4.94 - 3
y_start = (Y / H) * 4.48 - 2.24
x_end = (X / W) * 0.014 - 0.632
y_end = (Y / H) * 0.012 - 0.474

max_iter_start = 18

# Define colour palette
def setColour(t):
    T = 0.5
    if t < T:
        val = 256 * t / T
        return setPixel(0, 255 - val, 255)
    val = 256 * (t - T) / (1 - T)
    return setPixel(val, 0, 255 - val)

In [2]:
x0, y0, max_iteration = x_start, y_start, max_iter_start

!RepeatBlock(900)

x, y, iteration = 0, 0, 0
while ((x*x + y*y) <= 4) & (iteration < max_iteration):
    x, y = x*x - y*y + x0, 2*x*y + y0
    iteration += 1
setColour(iteration / max_iteration)

x0 += (x_end - x0) * 0.01
y0 += (y_end - y0) * 0.01
max_iteration *= 1.001

# Black Hat Snek
Graphcore have received an encrypted message, but have lost their super secure key for the decrypt function!

In [3]:
message = b'v\xa4q\x0f\xa8eA\xebp]\xb2$@\xbep\x0f\xbewF\xa5c\x0f\xff$f\x9bQ\\\xebmA\xebe\x0f\xa5k[\xaef@\xa4o\x0f\xadk]\xebb]\xaea\x0f\xa4j\x0f\x9be_\xaev\\\xbbeL\xae%'

def decrypt(key, data):
    return bytes(d ^ (((key % 0xfffff) >> (8 * (i % 3))) & 0xff) for i, d in enumerate(data))

But it's easy to exhaust a 2<sup>20</sup> search space when we have 2<sup>12</sup> python interpreters!

In [4]:
for key in range(__tileid, 1 << 20, __numtiles):
    result = decrypt(key, message)
    for crib in ('Graphcore', 'IPU', 'POD4'):
        if crib in result:
            print("Hit:", result)
            print("Key:", key)

-------[1x]-------
Hit: b'You can try out using 4 IPUs in a notebook for free on Paperspace!'
Key: 314159

-------[1x]-------
Hit: b'\x7fvp\x06zdH9qT`%Ilq\x06lvOwb\x06-%oIPU9lH9d\x06wjR|gIvn\x06\x7fjT9cT|`\x06vk\x06IdV|wUidE|$'
Key: 119305


We've also forgotten the password to our login function!

In [5]:
def login(X):
    masked_password = '@ufwodhub654'
    for x, y in zip(X, masked_password):
        if ord(x) ^ 7 != ord(y):
            return False
    return len(X) == len(masked_password)

We tried a timing attack (timing the login function to estimate how many characters we have correctly guessed) but system noise made it fail. If we try 5888 timing attacks, surely some will succeed?

In [6]:
password = ''
for _ in range(30):
    max_time = 0
    for c in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-_!:;~<>()*&^%$+=[]':
    
        t = getTime()
        _ = login(password + c)
        t = deltaTime(t, getTime())
            
        if t > max_time:
            best_c, max_time = c, t
        
    password += best_c
    if login(password):
        print('Found password:', password)
        break
else:
    print('Failed :(')

-------[5882x]-------
Failed :(

-------[6x]-------
Found password: Graphcore123
